In [0]:
!pip install -U -q PyDrive

In [2]:
!pip install hyperdash
from hyperdash import monitor_cell
!hyperdash signup --github

  Created wheel for hyperdash: filename=hyperdash-0.15.3-cp36-none-any.whl size=28553 sha256=1f9c13ad69796e657af0632bcc1d117ba3909a25ec56fe38773c4c9c63f31639
  Stored in directory: /root/.cache/pip/wheels/62/5f/af/bbcaeb6570e4904c14fb4c1b70fee559a3788182ce4d104ce7
Successfully built hyperdash
Opening browser, please wait. If something goes wrong, press CTRL+C to cancel.
 SSH'd into a remote machine, or just don't have access to a browser? Open this link in any browser and then copy/paste the provided access token: https://hyperdash.io/oauth/github/start?state=client_cli_manual 
Waiting for Github OAuth to complete.
If something goes wrong, press CTRL+C to cancel.
Access token: Flkufu3OzhqRiveWW5Psk+fhyLuWT/CTj42ryUJPT1I=
Successfully logged in! We also installed: yWgexKOerZf5n/R8HHG19FhvDS6ZndyExaBZ68fhmxM= as your default API key


In [3]:
from tensorflow.keras.callbacks import Callback
from hyperdash import Experiment

class Hyperdash(Callback):
    def __init__(self, entries, exp):
        super(Hyperdash, self).__init__()
        self.entries = entries
        self.exp = exp

    def on_epoch_end(self, epoch, logs=None):
        for entry in self.entries:
            log = logs.get(entry)            
            if log is not None:
                self.exp.metric(entry, log)

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!ls ./drive/My\ Drive/

 01.model
 amidaTest2.py
 bert-master.zip
 bert_x.npy
 cabocha-0.69.tar.bz2
'Colab Notebooks'
 C言語課題.gsite
 export
 fasttext-IMDB.bin
 fasttext-neologd.bin
 fasttext-normal.bin
 Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip
 ja.zip
 LBa.csv
 LBa.gsheet
 modified.pdf
 nmf.pdf
 Saitaihou.pdf
 train-val_pre.tsv
 train-val-pre.tsv
 train-val-small.tsv
 train-val-wakati-juman2.tsv
 train-val-wakati-juman-nva.tsv
 train-val-wakati-juman.tsv
 train-val-wakati-nva.tsv
 train-val-wakati.tsv
 vector_data
 六義園.gdoc
 無題のプレゼンテーション.gslides
 試作


In [0]:
%cp ./drive/My\ Drive/train-val-wakati-juman.tsv ./
%cp ./drive/My\ Drive/fasttext-normal.bin ./
%cp ./drive/My\ Drive/fasttext-neologd.bin ./

In [0]:
%mkdir model
%mkdir model/tuning

In [0]:
import os,sys
sys.path.append('../')

import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from typing import Callable, List, Optional, Tuple, Union
from sklearn.model_selection import learning_curve
from scipy import sparse
from scipy.sparse import load_npz
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import datetime
import logging

from sklearn.externals import joblib
from scipy.sparse import load_npz
from abc import ABCMeta, abstractmethod

from tqdm import tqdm
from keras.callbacks import EarlyStopping
from keras.layers import SpatialDropout1D, Bidirectional
from keras.layers.recurrent import LSTM, GRU
from keras.layers.advanced_activations import ReLU, PReLU
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

from gensim.models import KeyedVectors

class Util:
    @classmethod
    def dump(cls, value, path):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        joblib.dump(value, path, compress=True)

    @classmethod
    def load(cls, path):
        return joblib.load(path)

class Logger:
    def __init__(self):
        self.general_logger = logging.getLogger('general')
        self.result_logger = logging.getLogger('result')
        stream_handler = logging.StreamHandler()
        file_general_handler = logging.FileHandler('./model/general.log')
        file_result_handler = logging.FileHandler('./model/result.log')
        if len(self.general_logger.handlers) == 0:
            self.general_logger.addHandler(stream_handler)
            self.general_logger.addHandler(file_general_handler)
            self.general_logger.setLevel(logging.INFO)
            self.result_logger.addHandler(stream_handler)
            self.result_logger.addHandler(file_result_handler)
            self.result_logger.setLevel(logging.INFO)

    def info(self, message):
        # 時刻をつけてコンソールとログに出力
        self.general_logger.info('[{}] - {}'.format(self.now_string(), message))

    def result(self, message):
        self.result_logger.info(message)

    def result_ltsv(self, dic):
        self.result(self.to_ltsv(dic))

    def result_scores(self, run_name, scores):
        # 計算結果をコンソールと計算結果用ログに出力
        dic = dict()
        dic['name'] = run_name
        dic['score'] = np.mean(scores)
        for i, score in enumerate(scores):
            dic[f'score{i}'] = score
        self.result_ltsv(dic)

    def now_string(self):
        return str(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

    def to_ltsv(self, dic):
        return '\t'.join(['{}:{}'.format(key, value) for key, value in dic.items()])

class Model(metaclass=ABCMeta):
    def __init__(self, run_fold_name: str, params: dict) -> None:
        """コンストラクタ
        :param run_fold_name: ランの名前とfoldの番号を組み合わせた名前
        :param params: ハイパーパラメータ
        """
        self.run_fold_name = run_fold_name
        self.params = params
        self.model = None

    @abstractmethod
    def train(self, tr_x: np.array, tr_y: np.array,
              va_x: Optional[np.array] = None,
              va_y: Optional[np.array] = None) -> None:
        """モデルの学習を行い、学習済のモデルを保存する
        :param tr_x: 学習データの特徴量
        :param tr_y: 学習データの目的変数
        :param va_x: バリデーションデータの特徴量
        :param va_y: バリデーションデータの目的変数
        """
        pass

    @abstractmethod
    def predict(self, te_x: np.array) -> np.array:
        """学習済のモデルでの予測値を返す
        :param te_x: バリデーションデータやテストデータの特徴量
        :return: 予測値
        """
        pass

    @abstractmethod
    def score(self, te_x: np.array, te_y: np.array) -> float:
        """学習済のモデルでのスコア値を返す
        :te_x: np.array
        :te_y: np.array
        :return: 予測値
        """
        pass

    @abstractmethod
    def save_model(self, feature: str) -> None:
        """モデルの保存を行う"""
        pass

    @abstractmethod
    def load_model(self, feature: str) -> None:
        """モデルの読み込みを行う"""
        pass


logger = Logger()

class Runner:
    def __init__(self, run_name: str, model_cls: Callable[[str, dict], Model], features: str, params: dict):
        """コンストラクタ
        :param run_name: ランの名前
        :param model_cls: モデルのクラス
        :param features: 特徴量のリスト
        :param params: ハイパーパラメータ
        """
        self.run_name = run_name
        self.model_cls = model_cls
        self.features = features
        self.params = params
        self.train_x = None
        self.train_y = None
        self.test_x = None
        self.n_fold = 5

    def train_fold(self, i_fold: Union[int, str]) -> Tuple[
        Model, Optional[np.array], Optional[np.array], Optional[float]]:
        """クロスバリデーションでのfoldを指定して学習・評価を行う
        他のメソッドから呼び出すほか、単体でも確認やパラメータ調整に用いる
        :param i_fold: foldの番号（すべてのときには'all'とする）
        :return: （モデルのインスタンス、レコードのインデックス、予測値、評価によるスコア）のタプル
        """
        # 学習データの読込
        validation = i_fold != 'all'
        train_x = self.load_x_train()
        train_y = self.load_y_train()
        if validation:
            # 学習データ・バリデーションデータをセットする
            tr_idx, va_idx = self.load_index_fold(i_fold)
            tr_x, tr_y = train_x[tr_idx], train_y[tr_idx]
            va_x, va_y = train_x[va_idx], train_y[va_idx]
            
            # 学習を行う
            model = self.build_model(i_fold)
            model.train(tr_x, tr_y, va_x, va_y)

            # バリデーションデータへの予測・評価を行う
            va_pred = model.predict(va_x)
            score = model.score(va_x, va_y)

            # モデル、インデックス、予測値、評価を返す
            return model, va_idx, va_pred, score
        else:
            # 学習データ全てで学習を行う
            model = self.build_model(i_fold)
            model.train(train_x, train_y)

            # モデルを返す
            return model, None, None, None

    def train_fold_lr(self, i_fold: Union[int, str], lr_curve_train_sizes: List[int]) -> Tuple[
        Model, Optional[np.array], Optional[np.array], Optional[float]]:
        """クロスバリデーションでのfoldを指定して学習・評価を行う
        他のメソッドから呼び出すほか、単体でも確認やパラメータ調整に用いる
        :param i_fold: foldの番号（すべてのときには'all'とする）
        :return: （モデルのインスタンス、レコードのインデックス、予測値、評価によるスコア）のタプル
        """
        # 学習データの読込
        train_x = self.load_x_train()
        train_y = self.load_y_train()

        tr_idx, va_idx = self.load_index_fold(i_fold)
        # 学習を行う
        model = self.build_model(i_fold)
        
        tr_x, tr_y = train_x[tr_idx], train_y[tr_idx]
        va_x, va_y = train_x[va_idx], train_y[va_idx]
        
        tr_score = np.empty(len(lr_curve_train_sizes))
        val_score = np.empty(len(lr_curve_train_sizes))

        for i, n_train_samples in enumerate(lr_curve_train_sizes):
            model.train(tr_x[:n_train_samples], tr_y[:n_train_samples], va_x, va_y)
            tr_score[i] = model.score(tr_x[:n_train_samples], tr_y[:n_train_samples])
            val_score[i] = model.score(va_x, va_y)

        model.train(tr_x, tr_y, va_x, va_y)

        va_pred = model.predict(va_x)
        score = model.score(va_x, va_y)
        # モデル、インデックス、トレーニングスコア, バリデーションスコアを返す
        return model, va_idx, va_pred, score, tr_score, val_score 

    def run_train_cv(self, lr_curve_train_sizes: Optional[List[int]]=None) -> None:
        """クロスバリデーションでの学習・評価を行う
        学習・評価とともに、各foldのモデルの保存、スコアのログ出力についても行う
        """
        logger.info(f'{self.run_name} - start training cv')

        scores = []
        va_idxes = []
        preds = []

        lr_curve = not (lr_curve_train_sizes is None)

        if lr_curve:
            train_scores = np.empty(( 0, len(lr_curve_train_sizes) ) , float)
            valid_scores = np.empty(( 0, len(lr_curve_train_sizes) ) , float)
        # 各foldで学習を行う
        for i_fold in range(self.n_fold):
            # 学習を行う
            logger.info(f'{self.run_name} fold {i_fold} - start training')
            if lr_curve:
                model, va_idx, va_pred, score, tr_score, val_score  = self.train_fold_lr(i_fold, lr_curve_train_sizes)
                train_scores = np.append(train_scores, tr_score.reshape(1, len(lr_curve_train_sizes)) ,axis=0)
                valid_scores = np.append(valid_scores, val_score.reshape(1, len(lr_curve_train_sizes)),axis=0)
            else:
                model, va_idx, va_pred, score = self.train_fold(i_fold)
            logger.info(f'{self.run_name} fold {i_fold} - end training - score {score}')

            # モデルを保存する
            model.save_model(self.features)

            # 結果を保持する
            va_idxes.append(va_idx)
            scores.append(score)
            preds.append(va_pred)

        # 各foldの結果をまとめる
        va_idxes = np.concatenate(va_idxes)
        order = np.argsort(va_idxes)
        preds = np.concatenate(preds, axis=0)
        preds = preds[order]

        logger.info(f'{self.run_name} - end training cv - score {np.mean(scores)}')

        # 予測結果の保存
        Util.dump(preds, f'./model/pred/{self.features}/{self.run_name}-train.pkl')

        # 評価結果の保存
        logger.result_scores(self.run_name, scores)
        if lr_curve:
            self._plot_lr_curve(lr_curve_train_sizes, train_scores, valid_scores)
            

    def _plot_lr_curve(self, lr_curve_train_sizes : List[int], train_scores: np.array, valid_scores: np.array) -> None:
        plt.style.use('seaborn-whitegrid')
        train_mean = np.mean(train_scores, axis=0)
        train_std  = np.std(train_scores, axis=0)
        valid_mean = np.mean(valid_scores, axis=0)
        valid_std  = np.std(valid_scores, axis=0)   
        plt.plot(lr_curve_train_sizes, train_mean, color='orange', marker='o', markersize=5, label='lerning-curve')
        plt.fill_between(lr_curve_train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.1, color='orange')
        plt.plot(lr_curve_train_sizes, valid_mean, color='darkblue', marker='o', markersize=5,label='validation accuracy')
        plt.fill_between(lr_curve_train_sizes, valid_mean + valid_std,valid_mean - valid_std, alpha=0.1, color='darkblue') 
        plt.xlabel('#training samples')
        plt.ylabel('scores')
        plt.legend(loc='lower right')
        plt.savefig(f'./model/fig/learning-curve-{self.run_name}.png',dpi=300)


    def run_predict_cv(self) -> None:
        """クロスバリデーションで学習した各foldのモデルの平均により、テストデータの予測を行う
        あらかじめrun_train_cvを実行しておく必要がある
        """
        logger.info(f'{self.run_name} - start prediction cv')

        test_x = self.load_x_test()

        preds = []

        # 各foldのモデルで予測を行う
        for i_fold in range(self.n_fold):
            logger.info(f'{self.run_name} - start prediction fold:{i_fold}')
            model = self.build_model(i_fold)
            model.load_model(self.features)
            pred = model.predict(test_x)
            preds.append(pred)
            logger.info(f'{self.run_name} - end prediction fold:{i_fold}')

        # 予測の平均値を出力する
        pred_avg = np.mean(preds, axis=0)

        # 予測結果の保存
        Util.dump(pred_avg, f'./model/pred/{self.run_name}-test.pkl')

        logger.info(f'{self.run_name} - end prediction cv')

    def run_train_all(self) -> None:
        """学習データすべてで学習し、そのモデルを保存する"""
        logger.info(f'{self.run_name} - start training all')

        # 学習データ全てで学習を行う
        i_fold = 'all'
        model, _, _, _ = self.train_fold(i_fold)
        model.save_model(self.features)

        logger.info(f'{self.run_name} - end training all')

    def run_predict_all(self) -> None:
        """学習データすべてで学習したモデルにより、テストデータの予測を行う
        あらかじめrun_train_allを実行しておく必要がある
        """
        logger.info(f'{self.run_name} - start prediction all')

        test_x = self.load_x_test()

        # 学習データ全てで学習したモデルで予測を行う
        i_fold = 'all'
        model = self.build_model(i_fold)
        model.load_model(self.features)
        pred = model.predict(test_x)

        # 予測結果の保存
        Util.dump(pred, f'./model/pred/{self.run_name}-test.pkl')

        logger.info(f'{self.run_name} - end prediction all')

    def build_model(self, i_fold: Union[int, str]) -> Model:
        """クロスバリデーションでのfoldを指定して、モデルの作成を行う
        :param i_fold: foldの番号
        :return: モデルのインスタンス
        """
        # ラン名、fold、モデルのクラスからモデルを作成する
        run_fold_name = f'{self.run_name}-{i_fold}'
        return self.model_cls(run_fold_name, **self.params)

    def load_x_train(self) -> np.array:
        """学習データの特徴量を読み込む
        :return: 学習データの特徴量
        """
        # 学習データの読込を行う
        # 列名で抽出する以上のことを行う場合、このメソッドの修正が必要
        # 毎回train.csvを読み込むのは効率が悪いため、データに応じて適宜対応するのが望ましい（他メソッドも同様）
        if self.train_x is None:
            if self.features == "bow":
                self.train_x =load_npz('../vec/bow_train_x.npz').astype('float64')
            elif self.features == "n-gram":
                self.train_x =load_npz('../vec/n-gram_x.npz').astype('float64')
            elif self.features == "tf-idf":
                self.train_x = load_npz('../vec/tf-idf_x.npz').astype('float64')
            elif self.features == "n-gram-tf-idf":
                self.train_x =load_npz('../vec/n-gram-tf-idf_x.npz').astype('float64')
            elif self.features == "word2vec_mean":
                self.train_x = np.load('../vec/word2vec_pre_x_mean.npy', allow_pickle = True)
            elif self.features == "word2vec_max":
                self.train_x = np.load('../vec/word2vec_pre_x_max.npy', allow_pickle = True)
            elif self.features == "word2vec_concat":
                l = np.load('../vec/word2vec_pre_x_mean.npy', allow_pickle = True)
                r = np.load('../vec/word2vec_pre_x_max.npy', allow_pickle = True)
                self.train_x = np.hstack((l, r))
            elif self.features == "word2vec_hier":
                self.train_x = np.load('../vec/word2vec_pre_x_hier.npy', allow_pickle = True)
            elif self.features == "fasttext_mean":
                self.train_x = np.load('../vec/fasttext_x_mean.npy', allow_pickle = True)
            elif self.features == "fasttext_max":
                self.train_x = np.load('../vec/fasttext_x_max.npy', allow_pickle = True)
            elif self.features == "fasttext_concat":
                l = np.load('../vec/fasttext_x_mean.npy', allow_pickle = True)
                r = np.load('../vec/fasttext_x_max.npy', allow_pickle = True)
                self.train_x = np.hstack((l, r))
            elif self.features == "fasttext_hier":
                self.train_x = np.load('../vec/fasttext_x_hier.npy', allow_pickle = True)
            elif self.features == "doc2vec":
                self.train_x = np.load('../vec/doc2vec_pre_x.npy', allow_pickle=True)
            elif self.features == "scdv":
                matrix = np.load('../vec/word2vec_scdv_x.npy', allow_pickle=True)
                self.train_x = sparse.csr_matrix(matrix, dtype=np.float64)
            elif self.features == "bert":
                self.train_x = np.load('../vec/bert_x.npy', allow_pickle=True)
            elif self.features == "raw_text":
                df = pd.read_table("./train-val-wakati-juman.tsv", index_col=0)
                self.train_x = np.array(df["text"], dtype=str)
                self.train_y = np.array(df["label"], dtype=int)
        return self.train_x
    
    def load_y_train(self) -> np.array:
        """学習データの目的変数を読み込む
        :return: 学習データの目的変数
        """
        # 目的変数の読込を行う
        if self.train_y is None:
            if self.features == "bow_nva":
                self.train_y = np.load('../vec/bow_train_y_nva.npy', allow_pickle = True)
            else:
                self.train_y = np.load('../vec/y_full.npy', allow_pickle=True).astype('float64')
                
        return self.train_y

    def load_x_test(self) -> np.array:
        """テストデータの特徴量を読み込む
        :return: テストデータの特徴量
        """
        if self.test_x is None:
            if self.features == "bow":
                matrix = np.load('../vec/bow_test_x.npy', allow_pickle = True)
            elif self.features == "bow_nva":
                matrix = np.load('../vec/bow_test_x_nva.npy', allow_pickle = True)   
            self.test_x = sparse.csr_matrix(matrix, dtype=np.float64)
        return self.test_x
    
    def load_index_fold(self, i_fold: int) -> np.array:
        """クロスバリデーションでのfoldを指定して対応するレコードのインデックスを返す
        :param i_fold: foldの番号
        :return: foldに対応するレコードのインデックス
        """
        # 学習データ・バリデーションデータを分けるインデックスを返す
        # ここでは乱数を固定して毎回作成しているが、ファイルに保存する方法もある
        train_y = self.load_y_train()
        dummy_x = np.zeros(len(train_y))
        skf = StratifiedKFold(n_splits=self.n_fold, shuffle=True, random_state=71)
        return list(skf.split(dummy_x, train_y))[i_fold]

# tensorflowの警告抑制
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

class ModelGRU(Model):
    def __init__(self, run_fold_name, **params):
        super().__init__(run_fold_name, params)

    def train(self, tr_x, tr_y, va_x=None, va_y=None):
        """ 
            tr_x : List[str] (example.) [ "I am happy", "hello" ]
            tr_y : List[label]
            embedding_model : gensim.models.KeyedVectors Object
        """
        # scaling
        validation = va_x is not None

        # パラメータ
        nb_classes = 5
        embedding_dropout = self.params['embedding_dropout']
        gru_dropout = self.params['gru_dropout']
        gru_recurrent_dropout = self.params['recurrent_dropout']
        hidden_layers = int(self.params['hidden_layers'])
        hidden_units = int(self.params['hidden_units'])
        hidden_activation = self.params['hidden_activation']
        hidden_dropout = self.params['hidden_dropout']
        batch_norm = self.params['batch_norm']
        optimizer_type = self.params['optimizer']['type']
        optimizer_lr = self.params['optimizer']['lr']
        batch_size = int(self.params['batch_size'])
        nb_epoch = int(self.params['nb_epoch'])
        embedding_model = self.params['embedding_model']

        use_pre_embedding = not (embedding_model is None)

        # using keras tokenizer here
        self.token = Tokenizer(num_words=None)
        self.max_len = 70
        if validation:
            self.token.fit_on_texts(list(tr_x) + list(va_x))
        else:
            self.token.fit_on_texts(list(tr_x))

        xtrain_seq = self.token.texts_to_sequences(tr_x)
        tr_x = pad_sequences(xtrain_seq, maxlen=self.max_len)
        tr_y = np_utils.to_categorical(tr_y, num_classes=nb_classes)

        if validation:
            xvalid_seq = self.token.texts_to_sequences(va_x)
            va_x = pad_sequences(xvalid_seq, maxlen=self.max_len)
            va_y = np_utils.to_categorical(va_y, num_classes=nb_classes)

        word_index = self.token.word_index

        if use_pre_embedding:
            # create an embedding matrix
            vector_dim = embedding_model.vector_size
            embedding_matrix = np.zeros((len(word_index) + 1, vector_dim))
            for word, i in tqdm(word_index.items()):
                embedding_vector = embedding_model.wv[word]
                if embedding_vector is not None:
                    embedding_matrix[i] = embedding_vector
            
        self.model = Sequential()
        # input layer
        if use_pre_embedding:
            self.model.add(Embedding(
                    input_dim=len(word_index) + 1, 
                    output_dim=vector_dim,
                    input_length=self.max_len,
                    weights=[embedding_matrix],
                    trainable=False))
        else:
            self.model.add(Embedding(input_dim=len(word_index) + 1, 
                    output_dim=300,
                    input_length=self.max_len))

        self.model.add(SpatialDropout1D(embedding_dropout))
        self.model.add(GRU(300, dropout=gru_dropout, recurrent_dropout=gru_recurrent_dropout, return_sequences=True))
        self.model.add(GRU(300, dropout=gru_dropout, recurrent_dropout=gru_recurrent_dropout))
        # 中間層
        for i in range(hidden_layers):
            self.model.add(Dense(hidden_units))
            if batch_norm == 'before_act':
                self.model.add(BatchNormalization())
            if hidden_activation == 'prelu':
                self.model.add(PReLU())
            elif hidden_activation == 'relu':
                self.model.add(ReLU())
            else:
                raise NotImplementedError
            self.model.add(Dropout(hidden_dropout))

        # 出力層
        self.model.add(Dense(nb_classes, activation='softmax'))

        # オプティマイザ
        if optimizer_type == 'sgd':
            optimizer = SGD(lr=optimizer_lr, decay=1e-6, momentum=0.9, nesterov=True)
        elif optimizer_type == 'adam':
            optimizer = Adam(lr=optimizer_lr, beta_1=0.9, beta_2=0.999, decay=0.)
        else:
            raise NotImplementedError

        # 目的関数、評価指標などの設定
        self.model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # エポック数、アーリーストッピング
        # あまりepochを大きくすると、小さい学習率のときに終わらないことがあるので注意
        patience = 12
        # 学習の実行
        if validation:
            early_stopping = EarlyStopping(monitor='val_loss', patience=patience,
                                            verbose=1, restore_best_weights=True)
            history = self.model.fit(tr_x, tr_y, epochs=nb_epoch, batch_size=batch_size, verbose=2,
                                validation_data=(va_x, va_y), callbacks=[early_stopping])
        else:
            history = self.model.fit(tr_x, tr_y, nb_epoch=nb_epoch, batch_size=batch_size, verbose=2)

    def predict(self, te_x):
        xtest_seq = self.token.texts_to_sequences(te_x)
        te_x = pad_sequences(xtest_seq, maxlen=self.max_len)
        y_pred = self.model.predict(te_x)
        return y_pred

    def score(self, te_x, te_y):
        y_pred = self.predict(te_x)
        return f1_score(np.identity(5)[te_y], np.identity(5)[np.argmax(y_pred, axis=1)], average='samples')

    def save_model(self, feature):
        model_path = os.path.join(f'../model/model/{feature}', f'{self.run_fold_name}.h5')
        scaler_path = os.path.join(f'../model/model/{feature}', f'{self.run_fold_name}-scaler.pkl')
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        self.model.save(model_path)
        Util.dump(self.scaler, scaler_path)

    def load_model(self, feature):
        model_path = os.path.join(f'../model/model/{feature}', f'{self.run_fold_name}.h5')
        scaler_path = os.path.join(f'../model/model/{feature}', f'{self.run_fold_name}-scaler.pkl')
        self.model = load_model(model_path)
        self.scaler = Util.load(scaler_path)
        
        

In [0]:

params = {
        'embedding_dropout' : 0.3,
        'gru_dropout' : 0.3,
        'recurrent_dropout' : 0.3,
        'hidden_layers': 3,
        'hidden_units': 1024,
        'hidden_activation': 'relu',
        'hidden_dropout': 0.8,
        'batch_norm': 'before_act',
        'optimizer': {'type': 'adam', 'lr': 0.001},
        'batch_size': 500,
        'nb_epoch' : 100,
        'embedding_model' : None
    }
params['embedding_model'] = KeyedVectors.load_word2vec_format('./fasttext-neologd.bin', binary=True)
#params['embedding_model'] = KeyedVectors.load_word2vec_format('./fasttext-normal.bin', binary=True)
params_GRU = dict(params)

runner = Runner(run_name='GRU', model_cls=ModelGRU, features="raw_text", params=params_GRU)
runner.train_fold(0)

#runner.run_train_cv([ 100, 500, 1000, 2000, 5000, 6000, 7000, 8000])
#runner.run_predict_cv()